In [ ]:
#%matplotlib inline  
import powerlaw
import csv
import glob
from collections import defaultdict
import gc
import sys
from multiprocessing import Pool
import numpy as np

PUBYEAR_MIN = 1996
PUBYEAR_MAX = 2017

CITEYEAR_MAX = 2017

eid_info_dir = "../DATA/output/docinfo/eid_source_pubdate/"
eid_info_files = glob.glob(eid_info_dir + "*.txt")

pubyear_dict = {}
srcid_dict = {}
srcid_asjc = {}

_MAX_PROCESS = 8


def flatList(tdlist):
    templist = []
    for i in tdlist:
        templist += i
    return templist



In [ ]:
for nowfile in eid_info_files:
    with open(nowfile, 'rb') as csvfile: 
        reader = csv.reader(csvfile, delimiter='\t') 
        for row in reader: 
            if(row[1].isdigit()):
                pubyear_dict[row[0]] = int(row[1])
                srcid_dict[row[0]] = row[2]
gc.collect()

print len(pubyear_dict), len(srcid_dict)


In [ ]:
source_info_file = "../DATA/SCOPUS_SCIMAGO_MAPPING_DATA/src_final.txt"

with open(source_info_file, 'rb') as csvfile: 
    reader = csv.reader(csvfile, delimiter='\t') 
    for row in reader:
        srcid_asjc[row[0]] = row[5].split(";")
gc.collect()

In [ ]:
PArrayLength = PUBYEAR_MAX - PUBYEAR_MIN + 1
CArrayLength = CITEYEAR_MAX - PUBYEAR_MIN + 1

network_info_dir = "../DATA/output/network/separated/"
network_info_files = glob.glob(network_info_dir + "*.txt")
print len(network_info_files)

CountArray = []
for i in range(PArrayLength):
    temparray = []
    for j in range(CArrayLength):
        temparray.append({})
    CountArray.append(temparray)

SrcCountArray = []
for i in range(PArrayLength):
    temparray = []
    for j in range(CArrayLength):
        temparray.append({})
    SrcCountArray.append(temparray)

SrcPaperArray = []
for i in range(PArrayLength):
    SrcPaperArray.append({})    
    
ASJCCountArray = []
for i in range(PArrayLength):
    temparray = []
    for j in range(CArrayLength):
        temparray.append({})
    ASJCCountArray.append(temparray)

gc.collect()
count_exist = 0
for k, v in pubyear_dict.iteritems():
    if(k not in srcid_dict):
        continue
    elif(srcid_dict[k] not in srcid_asjc):
        continue
    elif(v > PUBYEAR_MAX or v < PUBYEAR_MIN):
        continue
    SrcPaperArray[v - PUBYEAR_MIN][srcid_dict[k]] = (SrcPaperArray[v - PUBYEAR_MIN]).setdefault(srcid_dict[k], 0) + 1        
    for nC in range(v - PUBYEAR_MIN, CArrayLength):
        count_exist += 1
        for asjcid in srcid_asjc[ srcid_dict[k] ]:
            ASJCCountArray[v - PUBYEAR_MIN][nC][asjcid] = 0
        CountArray[v - PUBYEAR_MIN][nC][k] = 0
        SrcCountArray[v - PUBYEAR_MIN][nC][srcid_dict[k]] = 0       

print "STEP2"
print count_exist
count_err = 0

for nowfile in network_info_files:
    with open(nowfile, 'rb') as csvfile: 
        reader = csv.reader(csvfile, delimiter='\t') 
        for row in reader: 
            if(row[0] not in srcid_dict or row[1] not in srcid_dict):
                pass
            elif(srcid_dict[row[0]] not in srcid_asjc or srcid_dict[row[1]] not in srcid_asjc):
                pass
            elif(pubyear_dict[row[0]] > PUBYEAR_MAX or pubyear_dict[row[0]] < PUBYEAR_MIN):
                pass
            elif(pubyear_dict[row[1]] > CITEYEAR_MAX):
                pass
            elif(pubyear_dict[row[1]] < pubyear_dict[row[0]]):
                count_err += 1 
            else:
                nP = pubyear_dict[row[0]] - PUBYEAR_MIN
                nC = pubyear_dict[row[1]] - PUBYEAR_MIN
                CountArray[nP][nC][row[0]] += 1
                for asjcid in srcid_asjc[ srcid_dict[row[0]] ]:
                    ASJCCountArray[nP][nC][asjcid] += 1
                SrcCountArray[nP][nC][srcid_dict[row[0]]] += 1                

for i in range(PArrayLength):
    for j in range(CArrayLength):
        for key, val in CountArray[i][j].iteritems():
            try:
                CountArray[i][j][key] = float(val*SrcPaperArray[i][srcid_dict[k]])/float(SrcCountArray[i][j][srcid_dict[key]])
            except ZeroDivisionError: CountArray[i][j][key] = 0

gc.collect()
print count_err            

In [ ]:
f_out = open('../DATA/distribution_compare_scrid_normalized/all_merged.txt', "w")

for i in range(PArrayLength):
    f_out.write("\n")
    for j in range(CArrayLength):
        data = sorted(CountArray[i][j].values(), reverse=True)
        if(len(data) < 2 or j < i):
            f_out.write("\t".join(str(x) for x in [i+PUBYEAR_MIN, j+PUBYEAR_MIN, len(data)])+ "\t-1\t0\t0\t0\t0\t0\t0\n")
            continue
        pl = powerlaw.Fit(data, descrete = True)
        likelihood_list = []
        likelihood_list.append(np.sum(pl.power_law.loglikelihoods(data)))
        likelihood_list.append(np.sum(pl.truncated_power_law.loglikelihoods(data)))
        likelihood_list.append(np.sum(pl.lognormal.loglikelihoods(data)))
        likelihood_list.append(np.sum(pl.lognormal_positive.loglikelihoods(data)))
        likelihood_list.append(np.sum(pl.exponential.loglikelihoods(data)))
        likelihood_list.append(np.sum(pl.stretched_exponential.loglikelihoods(data)))
        f_out.write("\t".join(str(x) for x in [i+PUBYEAR_MIN, j+PUBYEAR_MIN, len(data), likelihood_list.index(max(likelihood_list))])+"\t" 
                    + "\t".join(str(x) for x in likelihood_list) + "\n")
f_out.close()

In [ ]:
#%%capture
def PowerLawASJC(code):
    if(not code.isdigit()):
        return -1
    outfile = "../DATA/distribution_compare_scrid_normalized/ASJC_" + str(code) + ".txt"
    print outfile
    f_out = open(outfile, "w")
    for i in range(PArrayLength):
        f_out.write("\n")
        for j in range(CArrayLength):
            data = []
            for key, val in CountArray[i][j].iteritems():
                if(key not in srcid_dict):
                    continue
                elif(srcid_dict[key] not in srcid_asjc):
                    continue
                SourceID = srcid_dict[key]
                SourceASJC = srcid_asjc[SourceID]
                if(code in SourceASJC):
                    data.append(val)
            data = sorted(data, reverse=True)
            if(len(data) < 2 or j < i):
                f_out.write("\t".join(str(x) for x in [i+PUBYEAR_MIN, j+PUBYEAR_MIN, len(data)])+ "\t-1\t0\t0\t0\t0\t0\t0\n")
                continue
            pl = powerlaw.Fit(data, descrete = True)
            likelihood_list = []
            likelihood_list.append(np.sum(pl.power_law.loglikelihoods(data)))
            likelihood_list.append(np.sum(pl.truncated_power_law.loglikelihoods(data)))
            likelihood_list.append(np.sum(pl.lognormal.loglikelihoods(data)))
            likelihood_list.append(np.sum(pl.lognormal_positive.loglikelihoods(data)))
            likelihood_list.append(np.sum(pl.exponential.loglikelihoods(data)))
            likelihood_list.append(np.sum(pl.stretched_exponential.loglikelihoods(data)))
            f_out.write("\t".join(str(x) for x in [i+PUBYEAR_MIN, j+PUBYEAR_MIN, len(data), likelihood_list.index(max(likelihood_list))])+"\t" 
                        + "\t".join(str(x) for x in likelihood_list) + "\n")
    f_out.close()   
    return 0

oldstderr = sys.stderr
sys.stderr = open('log.txt', 'w')
               

print type(srcid_asjc.values())
listASJC = set(flatList(srcid_asjc.values()))

print listASJC 

p = Pool(processes=_MAX_PROCESS)
tempout = p.map(PowerLawASJC, listASJC)
p.close()
        
sys.stderr = oldstderr

In [ ]:
def PowerLawASJC2(code):
    if(not code.isdigit()):
        return -1
    outfile = "../DATA/distribution_compare_scrid_normalized/ASJC_" + str(code) + ".txt"
    print outfile
    f_out = open(outfile, "w")
    for i in range(PArrayLength):
        f_out.write("\n")
        for j in range(CArrayLength):
            data = []
            for key, val in CountArray[i][j].iteritems():
                if(key not in srcid_dict):
                    continue
                elif(srcid_dict[key] not in srcid_asjc):
                    continue
                SourceID = srcid_dict[key]
                SourceASJC = [x[:2] for x in srcid_asjc[SourceID]]
                if(code in SourceASJC):
                    data.append(val)
            data = sorted(data)
            if(len(data) < 2 or j < i):
                f_out.write("\t".join(str(x) for x in [i+PUBYEAR_MIN, j+PUBYEAR_MIN, len(data)])+ "\t-1\t0\t0\t0\t0\t0\t0\n")
                continue
            pl = powerlaw.Fit(data, descrete = True);
            likelihood_list = []
            likelihood_list.append(np.sum(pl.power_law.loglikelihoods(data)))
            likelihood_list.append(np.sum(pl.truncated_power_law.loglikelihoods(data)))
            likelihood_list.append(np.sum(pl.lognormal.loglikelihoods(data)))
            likelihood_list.append(np.sum(pl.lognormal_positive.loglikelihoods(data)))
            likelihood_list.append(np.sum(pl.exponential.loglikelihoods(data)))
            likelihood_list.append(np.sum(pl.stretched_exponential.loglikelihoods(data)))
            #print likelihood_list
            #print i+PUBYEAR_MIN, j+PUBYEAR_MIN, 
            f_out.write("\t".join(str(x) for x in [i+PUBYEAR_MIN, j+PUBYEAR_MIN, len(data), likelihood_list.index(max(likelihood_list))])+"\t" 
                        + "\t".join(str(x) for x in likelihood_list) + "\n")
    return 0

listASJC2 = set(x[:2] for x in flatList(srcid_asjc.values()))

p = Pool(processes=_MAX_PROCESS)
tempout = p.map(PowerLawASJC2, listASJC2)
p.close()

f_out.close()